In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data= pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
test_data=pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data.head(20)

In [ ]:
test_data.head()

In [ ]:
sample_submission = pd.read_csv('../input/sf-crime/sampleSubmission.csv.zip')
sample_submission.head(10)

In [ ]:
sample_submission.columns.value_counts().sum()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
train_data['year'] = train_data['Dates'].apply(lambda x : x.split()[0].split('-')[0])
train_data.head()

In [ ]:

train_data['Month'] = train_data['Dates'].apply(lambda x : x.split()[0].split('-')[1])
train_data['Day'] = train_data['Dates'].apply(lambda x : x.split()[0].split('-')[2])
time = train_data['Dates'].apply(lambda x : x.split()[1].split(':')[:2])
train_data.head()

In [ ]:
train_data['time'] = [int(x[0])*60 + int(x[1]) for x in time]
train_data.head()

In [ ]:
train_data.drop('Dates',axis = 1, inplace = True)
train_data.head()

In [ ]:
test_data['year'] = test_data['Dates'].apply(lambda x : x.split()[0].split('-')[0])
test_data['Month'] = test_data['Dates'].apply(lambda x : x.split()[0].split('-')[1])
test_data['Day'] = test_data['Dates'].apply(lambda x : x.split()[0].split('-')[2])
time = test_data['Dates'].apply(lambda x : x.split()[1].split(':')[:2])
test_data['time'] = [int(x[0])*60 + int(x[1]) for x in time]
test_data.head()

In [ ]:
test_data.drop('Dates',axis = 1, inplace = True)


In [ ]:
test_data.head()

In [ ]:
train_data['DayOfWeek'].value_counts()

In [ ]:
test_data['DayOfWeek'].value_counts()

In [ ]:
train_data=pd.concat([train_data,pd.get_dummies(train_data['DayOfWeek'])],axis=1)
train_data.drop('DayOfWeek',axis=1,inplace=True)
train_data.head()

# train_data['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],value=[i for i in range(0,7)],inplace=True)
# test_data['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],value=[i for i in range(0,7)],inplace=True)
# train_data.head()

In [ ]:
test_data=pd.concat([test_data,pd.get_dummies(test_data['DayOfWeek'])],axis=1)
test_data.drop('DayOfWeek',axis=1,inplace=True)
test_data.head()

In [ ]:
train_data = pd.concat([train_data,pd.get_dummies(train_data['PdDistrict'])],axis = 1)
train_data.drop('PdDistrict',axis = 1, inplace = True)
train_data.head()

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# lable_encoder = LabelEncoder()
# train_data['DayOfWeek'] = lable_encoder.fit_transform(train_data['DayOfWeek'])
# train_data.head()
# test_data['DayOfWeek'] = lable_encoder.fit_transform(test_data['DayOfWeek'])
# test_data.head()

In [ ]:
test_data.head()

In [ ]:
test_data = pd.concat([test_data,pd.get_dummies(test_data['PdDistrict'])],axis = 1)
test_data.drop('PdDistrict',axis = 1, inplace = True)
test_data.head()

In [ ]:
train_data.drop(['Descript','Resolution','Address'],axis = 1, inplace = True)
test_data.drop('Address',axis = 1, inplace = True)
print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
X = train_data.drop('Category',axis = 1)
y = pd.get_dummies(train_data['Category'])
y.head()

In [ ]:
X = X.astype(float)
X.head()

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
from tensorflow import keras
from keras.layers import Dense, Activation
from keras.models import Sequential


In [ ]:
#creating NW model
model=Sequential()
model.add(Dense(100,input_shape=(X.shape[1],)))
model.add(Dense(80,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(39,activation='softmax'))
model.summary()

In [ ]:
#compile model
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
train=model.fit(X_train,y_train, 
         batch_size=64,
         epochs=16,
         verbose=2,
         validation_data=(X_test,y_test))

In [ ]:
#drop id column from test data and convert all data to float type
test=test_data.drop(['Id'],axis=1)
test=test.astype(float)
test.info()

In [ ]:
sample_submission.head(10)

In [ ]:
sample_submission.drop('Id',axis=1)

In [ ]:
pred=model.predict(test)
print(pred)

In [ ]:
pred_final=[np.argmax(i) for i in pred]

In [ ]:
m_reshape = np.max(pred, axis=1).reshape(-1, 1)
pred_final = np.array((pred == m_reshape), dtype='int32')
pred_final

In [ ]:
s_sub_col=list(sample_submission.columns)
print(s_sub_col)

In [ ]:
s_sub_col.remove('Id')


In [ ]:
print(s_sub_col)

In [ ]:
submission = pd.DataFrame({'Id':test_data['Id']})
for i , entry in enumerate(s_sub_col):
    submission[entry] = pred_final[:,i]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('../working/submission.csv', index=False)